# Convert Hands CNN and Detector from ONNX to TFLite

### Hands CNN + Detector Conversion

In [ ]:
import sys
import tensorflow as tf
import onnx_tf
import onnx
import numpy as np

sys.path.append("D:\Programming\cybertruck")

In [ ]:
model_name = 'raw_cnn'

### Now Prepare the Onnx model for tensorflow and then convert to tflite

https://medium.com/@zergtant/convert-pytorch-model-to-tf-lite-with-onnx-tf-232a3894657c

In [ ]:
def convert_onnx_to_tflite(model_name):
    # Load  ONNX model
    onnx_model = onnx.load(f'{model_name}.onnx')

    # Convert ONNX model to TensorFlow format
    tf_model = onnx_tf.backend.prepare(onnx_model)
    # Export  TensorFlow  model 
    tf_model.export_graph(f'{model_name}.tf')

    # Then convert from TF to TFLite
    converter = tf.lite.TFLiteConverter.from_saved_model(f'{model_name}.tf')
    tflite_model = converter.convert()
    open(f'{model_name}.tflite', 'wb').write(tflite_model)

convert_onnx_to_tflite(model_name)

### Now test the converted model

https://www.tensorflow.org/lite/guide/inference
https://www.tensorflow.org/lite/guide/inference#run_inference_with_dynamic_shape_model

Note that by default, this model takes a batch size of 1. Which works out well since we want real time performance

In [ ]:
def test_tflite_model(model_name):
    model_filename = f"{model_name}.tflite"

    # Load the TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path=model_filename)
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Test the model on random input data.
    input_shape = input_details[0]['shape']
    print(input_shape)
    input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

    # The function `get_tensor()` returns a copy of the tensor data.
    # Use `tensor()` in order to get a pointer to the tensor.
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print(output_data)

test_tflite_model(model_name)

In [ ]:
def test_dynamic_batches_tflite(model_name):
    model_filename = f"{model_name}.tflite"

    # Load the TFLite model in TFLite Interpreter
    interpreter = tf.lite.Interpreter(model_path=model_filename)

    input_shape = interpreter.get_input_details()[0]['shape']
    input_shape[0] = 5 # Set to batch size of 5 instead of the default 1

    # Resize input shape for dynamic shape model and allocate tensor
    interpreter.resize_tensor_input(interpreter.get_input_details()[0]['index'], input_shape)
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    print(input_details) # Here we can see the input size has changed to (5, 3, 299, 299) 
    print(output_details) # The output has the corresponding changes

    input_data = np.array(np.random.random_sample((5, 3, 224, 224)), dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    print(output_data)

test_dynamic_batches_tflite(model_name)

# Converting CNNs to Pytorch Lite

In [1]:
import sys
import os
import torch
from torch.utils.mobile_optimizer import optimize_for_mobile


root_dir = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(root_dir)

from cnn.raw_cnn import Raw_CNN

/home/ron/Classes/CV-Systems/cybertruck
['/home/ron/Classes/CV-Systems/cybertruck/conversions', '/home/ron/miniconda3/envs/cyber/lib/python310.zip', '/home/ron/miniconda3/envs/cyber/lib/python3.10', '/home/ron/miniconda3/envs/cyber/lib/python3.10/lib-dynload', '', '/home/ron/.local/lib/python3.10/site-packages', '/home/ron/miniconda3/envs/cyber/lib/python3.10/site-packages', '/home/ron/Classes/CV-Systems/cybertruck']


In [2]:
raw_model = Raw_CNN(None).to('cuda')
x = torch.randn(5, 3, 299, 299).to('cuda')
raw_model.load_state_dict(torch.load(r"/home/ron/Classes/CV-Systems/cybertruck/cnn/raw_models/raw/SGD/epoch20_11-27_16:15:10_76acc.pt"))
raw_model.eval()


scripted_module = torch.jit.script(raw_model)
torchscript_model_optimized = optimize_for_mobile(scripted_module)
# Export lite interpreter version model (compatible with lite interpreter)
scripted_module._save_for_lite_interpreter("RAW_CNN.ptl")

/home/ron/miniconda3/envs/cyber/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name xception to current legacy_xception.
  model = create_fn(
